In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from Abalone_data_preprocessing import GANs_two_class_real_data
from choose_device import to_device
from fit import f1
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN


In [2]:

#print("This is train.py file")


def shuffle_in_unison(a, b):     #Shuffling the features and labels in unison.
    assert len(a) == len(b)       #In Python, the assert statement is used to continue the execute if the given condition evaluates to True.
    shuffled_a = np.empty(a.shape, dtype=a.dtype)       #Return a new array of given shape and type, without initializing entries.
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b


def Oversampled_data(X_train_SMOTE, y_train_SMOTE, y_train, device):
    X_oversampled_0 = []
    X_oversampled_2 = []
    X_oversampled_3 = []
    for i in y_train_SMOTE[(y_train.shape[0]):]:
        if i==0: 
            X_oversampled_0.append(X_train_SMOTE[i])
        elif i==2: 
            X_oversampled_2.append(X_train_SMOTE[i])
        elif i==3: 
            X_oversampled_3.append(X_train_SMOTE[i])
    X_oversampled_0 = torch.from_numpy(np.array(X_oversampled_0))
    X_oversampled_0 = to_device(X_oversampled_0.float(), device)
    X_oversampled_2 = torch.from_numpy(np.array(X_oversampled_2))
    X_oversampled_2 = to_device(X_oversampled_2.float(), device)
    X_oversampled_3 = torch.from_numpy(np.array(X_oversampled_3))
    X_oversampled_3 = to_device(X_oversampled_3.float(), device)

    return X_oversampled_0, X_oversampled_2, X_oversampled_3


In [3]:
def model_rf(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1_mes = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    return accuracy, f1_mes, precision, recall, model

In [4]:

df  = pd.read_csv('../standing_up_from_lying.csv')
print(df['Class'].value_counts())
# df.dropna(inplace=True)
minority = 1
majority = 0


Class
0    146499
1     18361
Name: count, dtype: int64


In [5]:
device = torch.device('cuda')

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Class'], axis=1), df['Class'], test_size=0.2, random_state=10)

#### Calculating train and test accuracy and f1 score of non oversampled training data ####
Normal_accuracy, Normal_f1_score, Normal_precision, Normal_recall, model_normal = model_rf(X_train, y_train, X_test, y_test) 
print("Normal_accuracy: ", Normal_accuracy)
print("Normal_f1_score: ", Normal_f1_score)
print("Normal_precision: ", Normal_precision)
print("Normal_recall: ", Normal_recall)
print(classification_report(y_test, model_normal.predict(X_test)))



Normal_accuracy:  0.9072546403008613
Normal_f1_score:  0.8907059144342152
Normal_precision:  0.8923306858640692
Normal_recall:  0.9072546403008613
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     29353
           1       0.68      0.30      0.41      3619

    accuracy                           0.91     32972
   macro avg       0.80      0.64      0.68     32972
weighted avg       0.89      0.91      0.89     32972



In [6]:

print("Before OverSampling, counts of label '0': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1))) 
# print("Before OverSampling, counts of label '2': {}".format(sum(y_train==2)))
# print("Before OverSampling, counts of label '3': {}".format(sum(y_train==3)))     


Before OverSampling, counts of label '0': 117146
Before OverSampling, counts of label '1': 14742


In [7]:
X_train_SMOTE,y_train_SMOTE = SMOTE().fit_resample(X_train,y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_SMOTE.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_SMOTE.shape))

print("After OverSampling, counts of label '0': {}".format(sum(y_train_SMOTE==0)))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_SMOTE==1))) 


After OverSampling, the shape of train_X: (234292, 6)
After OverSampling, the shape of train_y: (234292,) 

After OverSampling, counts of label '0': 117146
After OverSampling, counts of label '1': 117146


In [8]:
device = torch.device('cuda')

#### Calculating train and test accuracy and f1 score of SMOTE oversampled training data ####

Smote_accuracy, Smote_f1_score, Smote_precision, Smote_recall, model_smote = model_rf(X_train_SMOTE, y_train_SMOTE, X_test, y_test) 
print("Smote_accuracy: ", Smote_accuracy)
print("Smote_f1_score: ", Smote_f1_score)
print("Smote_precision: ", Smote_precision)
print("Smote_recall: ", Smote_recall)
print(classification_report(y_test, model_smote.predict(X_test)))


Smote_accuracy:  0.8823850539851995
Smote_f1_score:  0.8895408549201762
Smote_precision:  0.8998890369547263
Smote_recall:  0.8823850539851995
              precision    recall  f1-score   support

           0       0.95      0.91      0.93     29353
           1       0.47      0.63      0.54      3619

    accuracy                           0.88     32972
   macro avg       0.71      0.77      0.74     32972
weighted avg       0.90      0.88      0.89     32972



In [9]:
adasyn = ADASYN(sampling_strategy='minority', random_state=42, n_neighbors=5)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

In [10]:
print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_resampled.shape))

print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))
print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1))) 


After OverSampling, the shape of train_X: (234479, 6)
After OverSampling, the shape of train_y: (234479,) 

After OverSampling, counts of label '0': 117146
After OverSampling, counts of label '1': 117333


In [11]:
ADA_accuracy, ADA_f1_score, ADA_precision, ADA_recall, model_ADA = model_rf(X_resampled, y_resampled, X_test, y_test) 
print("ADA_accuracy: ", ADA_accuracy)
print("ADA_f1_score: ", ADA_f1_score)
print("ADA_precision: ", ADA_precision)
print("ADA_recall: ", ADA_recall)
print(classification_report(y_test, model_smote.predict(X_test)))

ADA_accuracy:  0.8683731651097901
ADA_f1_score:  0.8799727691960382
ADA_precision:  0.8988067803975504
ADA_recall:  0.8683731651097901
              precision    recall  f1-score   support

           0       0.95      0.91      0.93     29353
           1       0.47      0.63      0.54      3619

    accuracy                           0.88     32972
   macro avg       0.71      0.77      0.74     32972
weighted avg       0.90      0.88      0.89     32972



In [12]:
torch.cuda.is_available()

True

In [13]:
##################### TWO CLASS ABALONE #####################
##### Oversampled data from SMOTE that is now to be passed in SMOTified GANs #####
X_oversampled = X_train_SMOTE[(X_train.shape[0]):].to_numpy()
X_oversampled = torch.from_numpy(X_oversampled)
X_oversampled = to_device(X_oversampled.float(), device)


In [14]:
#print(X_oversampled.shape)
lr = 0.001
epochs = 150
batch_size = 128

X_real, y_real = GANs_two_class_real_data(X_train, y_train)   #Defining the real data to be put in GANs



c:\Users\Vinu's PC\Documents\GANclassimbalanced\SMOTified_GANs_code\Abalone_data_preprocessing.py:64: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()


<class 'pandas.core.frame.DataFrame'>


In [ ]:
#Training our SMOTified GANs and GANs model and fetching their trained generators.
generator_SG, generator_G = f1(X_train, y_train, X_train_SMOTE, y_train_SMOTE, X_real, y_real, X_oversampled, device, lr, epochs, batch_size, minority, majority)

Trained_X_oversampled_SG = generator_SG(X_oversampled.float().to(device)).cpu().detach().numpy()
Trained_SG_dataset = np.concatenate((X_train_SMOTE[:(X_train.shape[0])], Trained_X_oversampled_SG), axis=0)
X_trained_SG, y_trained_SG = shuffle_in_unison(Trained_SG_dataset, y_train_SMOTE)



  0%|          | 0/116 [00:00<?, ?it/s]

Epoch [1/150], loss_g: 6.4309, loss_d: 0.0070, real_score: 59.0329, fake_score: -4.2974


  0%|          | 0/116 [00:00<?, ?it/s]

Epoch [2/150], loss_g: 9.6669, loss_d: 0.0000, real_score: 51.2450, fake_score: -9.6389


  0%|          | 0/116 [00:00<?, ?it/s]

...

  0%|          | 0/116 [00:00<?, ?it/s]

Epoch [24/150], loss_g: 13.9317, loss_d: 0.0000, real_score: 129.5177, fake_score: -13.9262


  0%|          | 0/116 [00:00<?, ?it/s]

Epoch [25/150], loss_g: 13.9756, loss_d: 0.0000, real_score: 131.2292, fake_score: -13.9687


  0%|          | 0/116 [00:00<?, ?it/s]

In [16]:
#### Calculating train and test accuracy and f1 score of SMOTified GANs oversampled training data ####
# SG_test_accuracy, SG_train_accuracy, SG_f1_score = model_rf(X_trained_SG, y_trained_SG, X_test, y_test)


SG_accuracy, SG_f1_score, SG_precision, SG_recall, model_SG = model_rf(X_trained_SG, y_trained_SG, X_test, y_test)
print("SG_accuracy: ", SG_accuracy)
print("SG_f1_score: ", SG_f1_score)
print("SG_precision: ", SG_precision)
print("SG_recall: ", SG_recall)
print(classification_report(y_test, model_SG.predict(X_test)))



SG_accuracy:  0.7978723404255319
SG_f1_score:  0.7270552687901297
SG_precision:  0.6677844588344126
SG_recall:  0.7978723404255319
              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89        77
         1.0       0.00      0.00      0.00        17

    accuracy                           0.80        94
   macro avg       0.41      0.49      0.44        94
weighted avg       0.67      0.80      0.73        94



c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [17]:

GANs_noise = torch.randn((X_oversampled.shape[0]), (X_oversampled.shape[1]), device=device)
Trained_X_oversampled_G = generator_G(GANs_noise.float().to(device)).cpu().detach().numpy()
Trained_G_dataset = np.concatenate((X_train_SMOTE[:(X_train.shape[0])], Trained_X_oversampled_G), axis=0)
X_trained_G, y_trained_G = shuffle_in_unison(Trained_G_dataset, y_train_SMOTE)

In [18]:
#### Calculating train and test accuracy and f1 score of SMOTified GANs oversampled training data ####
# G_test_accuracy, G_train_accuracy, G_f1_score = test_model_lists(X_trained_G, y_trained_G.ravel(), X_test, y_test.ravel(), 30)



G_accuracy, G_f1_score, G_precision, G_recall, model_G = model_rf(X_trained_G, y_trained_G, X_test, y_test)
print("G_accuracy: ", G_accuracy)
print("G_f1_score: ", G_f1_score)
print("G_precision: ", G_precision)
print("G_recall: ", G_recall)
print(classification_report(y_test, model_G.predict(X_test)))


G_accuracy:  0.7872340425531915
G_f1_score:  0.7216312056737588
G_precision:  0.6661211129296235
G_recall:  0.7872340425531915
              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88        77
         1.0       0.00      0.00      0.00        17

    accuracy                           0.79        94
   macro avg       0.41      0.48      0.44        94
weighted avg       0.67      0.79      0.72        94



c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [19]:
#Calculate Model for ADA GANS

##### Oversampled data from ADA that is now to be passed in ADA GANs #####
X_oversampled = X_resampled[(X_train.shape[0]):].to_numpy()
X_oversampled = torch.from_numpy(X_oversampled)
X_oversampled = to_device(X_oversampled.float(), device)

In [20]:
#Training our SMOTified GANs and GANs model and fetching their trained generators.
generator_AG, generator_G = f1(X_train, y_train, X_resampled, y_resampled, X_real, y_real, X_oversampled, device, lr, epochs, batch_size, minority, majority)

Trained_X_oversampled_AG = generator_AG(X_oversampled.float().to(device)).cpu().detach().numpy()
Trained_AG_dataset = np.concatenate((X_resampled[:(X_train.shape[0])], Trained_X_oversampled_AG), axis=0)
X_trained_AG, y_trained_AG = shuffle_in_unison(Trained_AG_dataset, y_resampled)



  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/150], loss_g: 0.7269, loss_d: 0.8284, real_score: -0.5775, fake_score: -0.1241


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [2/150], loss_g: 0.7131, loss_d: 0.3479, real_score: 4.3601, fake_score: -0.0263


  0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch [148/150], loss_g: 8.3980, loss_d: 0.0001, real_score: 16.6016, fake_score: -8.3838


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [149/150], loss_g: 8.4115, loss_d: 0.0001, real_score: 16.6178, fake_score: -8.3975


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [150/150], loss_g: 8.4252, loss_d: 0.0001, real_score: 16.6357, fake_score: -8.4114


In [21]:

AG_accuracy, AG_f1_score, AG_precision, AG_recall, model_AG = model_rf(X_trained_AG, y_trained_AG, X_test, y_test)
print("AG_accuracy: ", AG_accuracy)
print("AG_f1_score: ", AG_f1_score)
print("AG_precision: ", AG_precision)
print("AG_recall: ", AG_recall)
print(classification_report(y_test, model_G.predict(X_test)))


AG_accuracy:  0.8085106382978723
AG_f1_score:  0.732415519399249
AG_precision:  0.6694120338595287
AG_recall:  0.8085106382978723
              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88        77
         1.0       0.00      0.00      0.00        17

    accuracy                           0.79        94
   macro avg       0.41      0.48      0.44        94
weighted avg       0.67      0.79      0.72        94



c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [22]:

print("Normal Accuracy: ",round(Normal_accuracy,3))
print("SMOTE Accuracy: ",round(Smote_accuracy,3))
print("G Accuracy: ",round(G_accuracy,3))
print("SG Accuracy: ",round(SG_accuracy,3))
print("ADA Accuracy: ",round(ADA_accuracy,3))
print("AG Accuracy: ",round(AG_accuracy,3))


print("Normal F1 Score: ",round(Normal_f1_score,3))
print("Normal Precision: ",round(Normal_precision,3))
print("Normal Recall: ",round(Normal_recall,3))

print("SMOTE F1 Score: ",round(Smote_f1_score,3))
print("SMOTE Precision: ",round(Smote_precision,3))
print("SMOTE Recall: ",round(Smote_recall,3))

print("G F1 Score: ",round(G_f1_score,3))
print("G Precision: ",round(G_precision,3))
print("G Recall: ",round(G_recall,3))

print("SG F1 Score: ",round(SG_f1_score,3))
print("SG Precision: ",round(SG_precision,3))
print("SG Recall: ",round(SG_recall,3))

print("ADA F1 Score: ",round(ADA_f1_score,3))
print("ADA Precision: ",round(ADA_precision,3))
print("ADA Recall: ",round(ADA_recall,3))

print("AG F1 Score: ",round(AG_f1_score,3))
print("AG Precision: ",round(AG_precision,3))
print("AG Recall: ",round(AG_recall,3))



Normal Accuracy:  0.819
SMOTE Accuracy:  0.798
G Accuracy:  0.787
SG Accuracy:  0.798
ADA Accuracy:  0.787
AG Accuracy:  0.809
Normal F1 Score:  0.738
Normal Precision:  0.671
Normal Recall:  0.819
SMOTE F1 Score:  0.743
SMOTE Precision:  0.719
SMOTE Recall:  0.798
G F1 Score:  0.722
G Precision:  0.666
G Recall:  0.787
SG F1 Score:  0.727
SG Precision:  0.668
SG Recall:  0.798
ADA F1 Score:  0.749
ADA Precision:  0.73
ADA Recall:  0.787
AG F1 Score:  0.732
AG Precision:  0.669
AG Recall:  0.809


In [23]:
print("NORMAL MODEL")
print(classification_report(y_test, model_normal.predict(X_test)))
print("ADA MODEL")
print(classification_report(y_test, model_ADA.predict(X_test)))
print("ADA GAN MODEL")
print(classification_report(y_test, model_AG.predict(X_test)))
print("SMOTE MODEL")
print(classification_report(y_test, model_smote.predict(X_test)))
print("GAN MODEL")
print(classification_report(y_test, model_G.predict(X_test)))
print("SMOTE GAN MODEL")
print(classification_report(y_test, model_SG.predict(X_test)))



NORMAL MODEL
              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90        77
         1.0       0.00      0.00      0.00        17

    accuracy                           0.82        94
   macro avg       0.41      0.50      0.45        94
weighted avg       0.67      0.82      0.74        94

ADA MODEL
              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88        77
         1.0       0.29      0.12      0.17        17

    accuracy                           0.79        94
   macro avg       0.56      0.53      0.52        94
weighted avg       0.73      0.79      0.75        94

ADA GAN MODEL
              precision    recall  f1-score   support

         0.0       0.82      0.99      0.89        77
         1.0       0.00      0.00      0.00        17

    accuracy                           0.81        94
   macro avg       0.41      0.49      0.45        94
weighted avg       0.67      0.81    

c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\U

In [24]:
def hellinger(p, q):
    return np.sqrt(0.5 * ((np.sqrt(p) - np.sqrt(q)) ** 2).sum())

def hellingerDistance(train, generated):
    df1 = pd.DataFrame(train)
    df2 = pd.DataFrame(generated)
    df2.columns = df1.columns
    
    # print(df1['radius_0ean'].count())
    # print(df2['radius_0ean'].count())
    
    common = pd.merge(df1, df2, how='inner')
    # print(common['radius_0ean'].count())
    
    df2 = pd.concat([df2, common]).drop_duplicates(keep=False)
    # print(df2['radius_0ean'].count())
    
    df1 = df1.div(df1.sum(), axis=1)
    df2 = df2.div(df2.sum(), axis=1)
    
    # print((df2.head()))
    hellinger_distances = {}
    for col in df1.columns:
        dist = hellinger(df1[col], df2[col])
        hellinger_distances[col] = dist
    
    # print(hellinger_distances)
    average_distance = np.mean(list(hellinger_distances.values()))
    return average_distance



    

In [25]:
minorityTrainData = X_train[y_train == minority]
minorityTrainData.shape

(53, 16)

In [26]:
print("Hellinger Distance with SMOTE: ", hellingerDistance(minorityTrainData, X_train_SMOTE))
print("Hellinger Distance with GAN: ", hellingerDistance(minorityTrainData, X_trained_G))

X_ADA_CPU = [t.cpu() for t in X_oversampled]
X_ADA_CPU = torch.stack(X_ADA_CPU)


print("Hellinger Distance with ADA: ", hellingerDistance(minorityTrainData, X_ADA_CPU))
print("Hellinger Distance with ADA GAN: ", hellingerDistance(minorityTrainData, X_trained_AG))
print("Hellinger Distance with SGAN: ", hellingerDistance(minorityTrainData, X_trained_SG))


Hellinger Distance with SMOTE:  0.5132727616594642
Hellinger Distance with GAN:  0.5343256594547294
Hellinger Distance with ADA:  0.34983300484983676


C:\Users\Vinu's PC\AppData\Local\Temp\ipykernel_12004\2720793446.py:12: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  common = pd.merge(df1, df2, how='inner')
C:\Users\Vinu's PC\AppData\Local\Temp\ipykernel_12004\2720793446.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pd.concat([df2, common]).drop_duplicates(keep=False)
C:\Users\Vinu's PC\AppData\Local\Temp\ipykernel_12004\2720793446.py:12: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  common = pd.merge(df1, df2, how='inner')


Hellinger Distance with ADA GAN:  0.5556458817199805
Hellinger Distance with SGAN:  0.577100570557678


C:\Users\Vinu's PC\AppData\Local\Temp\ipykernel_12004\2720793446.py:12: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  common = pd.merge(df1, df2, how='inner')
